**Data Science and AI for Energy Systems** 

Karlsruhe Institute of Technology

Institute of Automation and Applied Informatics

Summer Term 2024

---

# Exercise II: Energy System Modeling - Solution

**Imports**

In [ ]:
import pypsa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('bmh')
%matplotlib inline

## Problem II.2 (programming) - Storage Optimization with PyPSA

Python for Power System Analysis (PyPSA) is a free software toolbox for optimising modern power systems that include features such as variable wind and solar generation, storage units, etc..

Use the toolbox to extend on your findings in Problem II.1.

***
**(a) Build a network in PyPSA with the two buses North and South and attach the load at each bus and attach the wind and solar generators with availability according to $g^N_w(t) = c_w(1+A_w\sin \omega_w t)$ and $g^S_s(t) = c_s(1+A_s\sin \omega_s t)$ for a year (you have to call `network.set_snapshots` to select a year) and with `p_nom_extendable=True`.**

> **Remarks:** For time reasons, you do not have to build the network from scratch. However, to get you acquainted with PyPSA we have omitted a few elements or some of the parameters of the network marked by three question marks `???`. Either, you have to add an element similar to the one in the box above or add a few parameters.

In [ ]:
network = pypsa.Network()

Add North and South bus

In [ ]:
network.add("Bus",
            "North",
            carrier="AC")

In [ ]:
network.add("Bus",
            "South",
            carrier="AC")

Attach constant load

In [ ]:
network.add("Load",
            "North Load",
            bus="North",
            p_set=20e3)

In [ ]:
network.add("Load",
            "South Load",
            bus="South",
            p_set=30e3)

Attach renewable generators according to given parameters

In [ ]:
# use a month instead of a year in this exercise for faster optimization
network.set_snapshots(np.arange(0, 4*7*24))
# network.set_snapshots(np.arange(0, 365*24))

In [ ]:
Cfw = 0.3
Aw = 0.9
omegaw = 2*np.pi/(7*24)

Cfs = 0.12
As = 1.
omegas = 2*np.pi/24

GNwt = Cfw * (1+Aw*np.sin(omegaw*network.snapshots.to_series()))
GSst = Cfs * (1+As*np.sin(omegas*network.snapshots.to_series()))

In [ ]:
pd.concat([GNwt, GSst], keys=['wind', 'solar'], axis=1).loc[:4*7*24].plot()

In [ ]:
network.add("Generator",
            "Wind",
            bus="North",
            p_nom_extendable=True,
            capital_cost=1.2e6,
            p_max_pu=GNwt)

In [ ]:
network.add("Generator",
            "Solar",
            bus="South",
            p_nom_extendable=True,
            capital_cost=0.6e6,
            p_max_pu=GSst)

***
**(b) Attach extendable storage units at the North and the South! The storages have to be modelled as an `H2-bus` (a bus with `carrier='H2'`) linked to the `AC-bus` North with a `Link` where `p_nom_extendable=True` with the `capital_cost` (in currency/MW) of the power capacity and an also extendable `Store` with the `capital_cost` (in currency/MWh) of the energy capacity, for instance. The losses can be set on the links as `efficiency`.**

In [ ]:
for bus in ["North", "South"]:
    
    # H2 storage
    network.add("Bus",
                bus + " H2",
                carrier="H2")
    
    network.add("Store",
                bus + " H2 St.",
                bus=bus + " H2",
                e_nom_extendable=True,
                capital_cost=10e3)
    
    network.add("Link",
                bus + "->H2",
                bus0=bus, bus1=bus + " H2",
                p_nom_extendable=True,
                capital_cost=0.3e6,
                efficiency=0.75)
    
    network.add("Link",
                "H2->" + bus,
                bus0=bus + " H2", bus1=bus,
                p_nom_extendable=True,
                capital_cost=0.45e6,
                efficiency=0.405)
    
    # Battery storage
    network.add("Bus",
                bus + " Battery",
                carrier="Battery")
    
    network.add("Store",
                bus + " Battery St.",
                bus=bus + " Battery",
                e_nom_extendable=True,
                capital_cost=0.2e6)
    
    network.add("Link",
                bus + "->Battery",
                bus0=bus, bus1=bus + " Battery",
                p_nom_extendable=True,
                capital_cost=0.15e6,
                efficiency=0.9)
    
    network.add("Link",
                "Battery->" + bus,
                bus0=bus + " Battery", bus1=bus,
                p_nom_extendable=True,
                capital_cost=0.15e6,
                efficiency=0.9)

In [ ]:
network

***
**(c) Run an investment optimization by calling the `optimize` function.**

In [ ]:
network.optimize()

***
**(d) How do your results `objective` and `{generators,stores,links}.p_nom_opt` compare with the results of III.1(d)?** 

Objective value

In [ ]:
obj_v1 = network.objective / 1e9 # Mio. Euro
obj_v1

In [ ]:
network.generators

Capacities for wind and solar.

In [ ]:
res_cap_v1 = network.generators.p_nom_opt / 1e3 # GW
res_cap_v1 

Store and dispatch power capacity.

In [ ]:
sto_cap_v1 = network.links.p_nom_opt / 1e3 # GW
sto_cap_v1 

Energy capacities.

In [ ]:
sto_engy_v1 = network.stores.e_nom_opt / 1e6 # TWh
sto_engy_v1

Plot the storage energy states over time

In [ ]:
network.stores_t.e.plot()

***
**(e) Now we lift the restriction against transmission and allow North and South to bridge their 500 km
separation with a transmission line. How does the cost optimal technology mix change?**

Add extendable link between North and South: 

In [ ]:
network.add("Link",
            "North<->South",
            bus0="North", bus1="South",
            p_min_pu=-1,
            p_nom_extendable=True,
            capital_cost=0.2e6)

Run optimization:

In [ ]:
# network.lopf(solver_name=solver)
network.optimize()

Get the results `objective` and `{generators,stores,links}.p_nom_opt` with real availability:

In [ ]:
obj_v2 = network.objective / 1e9 # Mio. Euro
obj_v2

In [ ]:
# (a) Capacities for wind and solar.
res_cap_v2 = network.generators.p_nom_opt / 1e3  # GW
res_cap_v2

In [ ]:
# (b) Store and dispatch power capacity.
sto_cap_v2 = network.links.p_nom_opt / 1e3 # GW
sto_cap_v2

In [ ]:
# (c) Energy capacities
sto_engy_v2 = network.stores.e_nom_opt / 1e6 # TWh
sto_engy_v2

In [ ]:
network.stores_t.e.plot()

***
**(f) Replace the approximated availability time-series of the wind and the solar generators with the ones from `availability.csv` computed from reanalysis weather data and re-run the LOPF. Compare the results! Explain the differences by looking at the cumulative variations relative to the mean of the availability time-series!**

Adapt the network to new availabiltiy data:

In [ ]:
network.remove("Generator", "Wind")
network.remove("Generator", "Solar")

In [ ]:
availability = pd.read_csv("data/availability.csv", index_col=0, parse_dates=True)
availability.head()

In [ ]:
availability.loc["2012-7"].plot()

In [ ]:
availability.loc["2012-7"].index

In [ ]:
network.set_snapshots(availability.loc["2012-7"].index)

In [ ]:
network.add("Generator",
            "Wind",
            bus="North",
            p_nom_extendable=True,
            capital_cost=1.2e6,
            p_max_pu=availability["wind"])

In [ ]:
network.add("Generator",
            "Solar",
            bus="South",
            p_nom_extendable=True,
            capital_cost=0.6e6,
            p_max_pu=availability["solar"])

Run optimization:

In [ ]:
network.optimize()

Get the results `objective` and `{generators,stores,links}.p_nom_opt` with real availability:

In [ ]:
obj_v3 = network.objective / 1e9 # Mio. Euro
obj_v3

In [ ]:
# (a) Capacities for wind and solar.
res_cap_v3 = network.generators.p_nom_opt / 1e3
res_cap_v3

In [ ]:
# (b) Store and dispatch power capacity.
sto_cap_v3 = network.links.p_nom_opt / 1e3
sto_cap_v3

In [ ]:
# (c) Energy capacities
sto_engy_v3 = network.stores.e_nom_opt / 1e6
sto_engy_v3

In [ ]:
network.stores_t.e.plot()

In [ ]:
np.cumsum(availability.loc["2012-7"] - availability.loc["2012-7"].mean()).plot()

***
**(g) Compare all results for all three scenarios in terms of total system cost, renewable generation capacity, storage power capacity and storage energy capacity!**

> **Remark:** For example, you can use bar charts `plt.bar(...)` or `df.plot.bar()` to visualize the differences.

In [ ]:
scens = ["without transmission",
         "with transmission",
         "with real data and transmission"]

attrs = ["storage power capacity",
         "storage energy capacity",
         "renewable capacity"]

In [ ]:
sto_cap_v1.name = attrs[0] + " " + scens[0]
sto_cap_v2.name = attrs[0] + " " + scens[1]
sto_cap_v3.name = attrs[0] + " " + scens[2]

res_cap_v1.name = attrs[1] + " " + scens[0]
res_cap_v2.name = attrs[1] + " " + scens[1]
res_cap_v3.name = attrs[1] + " " + scens[2]

sto_engy_v1.name = attrs[2] + " " + scens[0]
sto_engy_v2.name = attrs[2] + " " + scens[1]
sto_engy_v3.name = attrs[2] + " " + scens[2]

In [ ]:
values = [obj_v1, obj_v2, obj_v3]
plt.bar(scens,values)
plt.ylabel('Mio. Euro')
plt.show()

In [ ]:
sto_caps = pd.concat([sto_cap_v1,sto_cap_v2,sto_cap_v3], axis=1, sort=False)
sto_caps.plot.bar()

In [ ]:
res_caps = pd.concat([res_cap_v1,res_cap_v2,res_cap_v3], axis=1, sort=False)
res_caps.plot.bar()

## Problem II.3 (programming) - Meshed AC-DC Network

We now turn to a slightly bigger, 3-node AC network coupled via AC-DC converters to a
3-node DC network. There is also a single point-to-point DC using the Link component.

Load the network

In [ ]:
network = pypsa.examples.ac_dc_meshed(from_master=True)

In [ ]:
# get current type (AC or DC) of the lines from the buses
lines_current_type = network.lines.bus0.map(network.buses.carrier)
lines_current_type

In [ ]:
network.links.loc["Norwich Converter", "p_nom_extendable"] = False

***
**(a) First inspect the topology of the network using the
`determine_network_topology` and the
`network.sub_networks` functions.**

In [ ]:
network.determine_network_topology()
network.sub_networks["n_branches"] = [
    len(sn.branches()) for sn in network.sub_networks.obj
]
network.sub_networks["n_buses"] = [len(sn.buses()) for sn in network.sub_networks.obj]

network.sub_networks

***
**(b) Check out the six generators in this network and sort them
by their capital and marginal costs**

In [ ]:
network.generators.sort_values(by='capital_cost')

In [ ]:
network.generators.sort_values(by='marginal_cost')

We see that the generators have different capital and marginal costs. All of them have a `p_nom_extendable` set to `True`, meaning that capacities can be extended in the optimization.

***
**(c) Plot the per unit limit for the wind generators at each time
step, given by the weather potentials at the site.**

You can find the relevant data in `network.generators_t.p_max_pu`

In [ ]:
network.generators_t.p_max_pu.plot.area(subplots=True)
plt.tight_layout()

***
**(d) Now we know what the network looks like and where the
generators and lines are. Now Perform an optimization
of the operation and capacities. What are the optimized
system costs?**

In [ ]:
network.optimize()

In [ ]:
network.objective

Why is this number negative? It considers the starting point of the optimization, thus the existent capacities given by `network.generators.p_nom` are taken into account.

The real system cost are given by

In [ ]:
network.objective + network.objective_constant

***
**(e) Check the optimal generator capacities and their production time series.**

The optimal capacities are given by `p_nom_opt` for generators, links and storages and `s_nom_opt` for lines.

In [ ]:
network.generators.p_nom_opt.div(1e3).plot.bar(ylabel="GW", figsize=(8, 3))
plt.tight_layout()

Their production is again given as a time-series in `network.generators_t.p`.


In [ ]:
network.generators_t.p.div(1e3).plot.area(subplots=True, ylabel="GW")
plt.tight_layout()

***
**(f) Plot the marginal prices at each location.**

The marginal prices for each bus can be found in `network.buses_t.marginal_price`.

In [ ]:
network.buses_t.marginal_price.plot.area(subplots=True, ylabel="€/MWh")
plt.tight_layout()